The purpose of this notebook is to get the head position and direction of the ferret for each stimulus presentation.

The input data consists of two sources:
(1) Stimulus metadata after temporal alignment to both video and neural recording systems
(2) LED tracking data

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

In [7]:
data_dir = Path('/home/stephen/Github/Speaker_Grid/data/F1901_Crumble_Squid/2021-05-27_Squid_15-57')

stim_file = '2021-05-27T17-57-07_StimData_MCSVidAlign.csv'
tracking_file = '2021-05-27_SquidVid_17_56_08.csv'

stim = pd.read_csv( str(data_dir / stim_file))

print(stim.shape)

LEDs = pd.read_csv( str(data_dir / tracking_file))

print(LEDs.shape)

(829, 10)
(18345, 14)


In [11]:
join_data = pd.merge( stim, LEDs, left_on='closest_frame', right_index=True)

print(join_data.shape)

(829, 24)


### Tracking data: Problematic positions

We know that when the LEDs go out of view, the tracking code returns ridiculous values. Here we replace anything that isn't sensible with nans

In [17]:


join_data.loc[join_data['red_x'] < 150, 'red_x'] = np.nan


In [18]:
join_data['head_x'] = (join_data['blue_x'] + join_data['red_x']) / 2

In [19]:
join_data[['head_x','blue_x','red_x']].head(n=20)


,head_x,blue_x,red_x
0,323.310369,317.981450,328.639288
1,324.682402,318.601083,330.763721
2,324.526259,318.309560,330.742959
3,324.755314,318.678372,330.832257
4,453.681295,317.878278,589.484312
5,1646.358905,317.890528,2974.827282
6,618.163383,317.902911,918.423855
7,323.018656,317.811995,328.225318
8,517.283159,317.107952,717.458365
9,324.713919,318.552258,330.875581
